In [66]:
import pandas as pd
import requests
import lxml
from lxml import html
from datetime import datetime
from sqlalchemy import create_engine, text, exc
import logging
import psycopg2
from psycopg2.extras import execute_values

### XPath

In [67]:
url = "https://www.kinoafisha.info/releases/"
r = requests.get(url)

In [68]:
tree = html.fromstring(r.content)

In [69]:
releases_xpath = tree.xpath('//div[@class="movieList_title grid_cell12"]')

In [70]:
list_relise_xpath = tree.xpath('//div[@class="releases_list movieList movieList-grid grid"]')[0]
list_movies_xpath = list_relise_xpath.xpath('.//div[@class="movieList_item movieItem  movieItem-grid grid_cell3  "]')

In [73]:
headers = {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 18_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.5 Mobile/15E148 Safari/604.1"}

In [74]:
films_xpath = []
for movies in list_movies_xpath:
    href_element = movies.xpath('.//a[@class="movieItem_title"]')[0]
    title = href_element.text_content().strip()
    ref1 = href_element.get('href').replace('\n', '')
    
    r = requests.post(ref1, headers)
    tree = html.fromstring(r.content)
    
    description_elem = tree.xpath('//span[@class="newFilmInfo_description"]')
    description = description_elem[0].text_content().strip() if description_elem else ''
    
    title_secondary_elem = tree.xpath('//span[@class="newFilmInfo_titleSecondary"]')
    title_secondary = title_secondary_elem[0].text_content().strip() if title_secondary_elem else ''
    
    age_elem = tree.xpath('//span[@class="newFilmInfo_age"]')
    age = age_elem[0].text_content().strip() if age_elem else ''
    
    genre_elem = tree.xpath('//nav[@class="newFilmInfo_genreMenu swipe outer-mobile inner-mobile"]')
    if genre_elem:
        genre_texts = [text.strip() for text in genre_elem[0].xpath('.//text()') if text.strip()]
        genre = ', '.join(genre_texts)
    else:
        genre = ''
    
    producer_elem = tree.xpath('//span[@class="badgeList_name"]')
    producer = producer_elem[0].text_content().strip() if producer_elem else ''
    
    actors_elem = tree.xpath('//div[@class="filmInfoActors_item filmInfoActors_item-actors"]')
    if actors_elem:
        actors_columns = actors_elem[0].xpath('.//div[@class="filmInfoActors_item_columns"]')
        if actors_columns:
            actor_texts = [text.strip() for text in actors_columns[0].xpath('.//text()') if text.strip()]
            actors = ', '.join(actor_texts)
        else:
            actors = ''
    else:
        actors = ''
    
    rating_elem = tree.xpath('//div[@class="ratingBlockCard_local"]')
    rating = rating_elem[0].text_content().strip() if rating_elem else '0.0'
    
    info_films_elem = tree.xpath('//div[@class="newFilmInfo_info swipe outer-mobile inner-mobile"]')
    if info_films_elem:
        info_texts = [text.strip() for text in info_films_elem[0].xpath('.//text()') if text.strip()]
        info_films = ', '.join(info_texts)
    else:
        info_films = ''
    
    films_xpath.append([title, title_secondary, description, rating, age, genre, producer, actors, info_films])

films_xpath_df = pd.DataFrame(films_xpath, columns=['title', 'title_secondary', 'description', 'rating', 'age', 'genre', 'producer', 'actors', 'info_films'])

In [75]:
films_xpath_df['rating'] = films_xpath_df['rating'].astype(float)

In [76]:
films_xpath_df

,title,title_secondary,description,rating,age,genre,producer,actors,info_films
0,Специалист,Dhurandhar,Загадочный путник внедряется в индийскую мафию,0.0,,"боевик, приключения, драма",Адитья Дхар,"Ранвир Сингх, Арджун Рампал, Мадхаван, Санджай...","Страна, Индия, Продолжительность, 3 часа 32 ми..."
1,Зверополис 2,Zootopia 2,"Джуди и Ник вновь объединяются, чтобы раскрыть...",8.8,6+,"приключения, анимация, комедия",Джаред Буш,"Джейсон Бэйтман, Кинта Брансон, Фортун Феймсте...","Страна, США, Продолжительность, 1 час 47 минут..."
2,Пять ночей с Фредди 2,Five Nights at Freddy's 2,Пиццерия Freddy Fazbear's снова открыта. Продо...,7.9,18+,"ужасы, детектив, триллер",Эмма Тамми,"Мэттью Лиллард, Пайпер Рубио, Меган Фокс, Джош...","Страна, США, Продолжительность, 1 час 44 минут..."
3,Агент времени: Глава Инду. Фильм,Link Click: Bridon Arc - The Movie,Полнометражное продолжение популярного китайск...,8.4,18+,"приключения, анимация, криминал",Haolin Li,,"Страна, Китай, Продолжительность, 1 час 41 мин..."
4,Невероятные приключения Шурика,Neveroyatnye priklyucheniya Shurika,Комики телеканала ТНТ в новом пародийном фильм...,4.4,18+,"комедия, приключения, музыка",Роман Ким,"Тимур Батрутдинов, Марина Кравец, Гарик Мартир...","Страна, Россия, Продолжительность, 2 часа 3 ми..."
...,...,...,...,...,...,...,...,...,...
145,"Винни-Пух и все,все,все",,,0.0,,семейный,,,"Страна, Россия, Год выпуска, 2027, Премьера в ..."
146,Весельчак У,,,0.0,,"фэнтези, семейный",Александр Андрющенко,"Александр Петров, Анастасия Талызина, Мирон Пр...","Страна, Россия, Год выпуска, 2027, Премьера в ..."
147,Ханна,,,0.0,,"военный, драма",Иван Коровин,"Екатерина Симаходская, Людмила Артемьева, Дани...","Страна, Россия, Год выпуска, 2027, Премьера в ..."
148,Сивка-бурка,Sivka-burka,,0.0,6+,семейный,Александр Амиров,,"Страна, Россия, Год выпуска, 2026, Премьера в ..."


### сохранение данных в postgresql

In [77]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [79]:
POSTGRES_CONFIG = {
    'user': 'admin',
    'password': None,
    'host': 'localhost',
    'port': '5432',
    'database': 'admin'
}

connection_string = f"postgresql://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}@{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['database']}"

engine = create_engine(connection_string)

films_xpath_df.to_sql(
    'movies',         
    engine,          
    if_exists='replace',  
    index=False,     
    chunksize=1000,    
    method='multi'   
)

print(f"Данные успешно сохранены! Записей: {len(films_xpath_df)}")

Данные успешно сохранены! Записей: 150


функция для добавления новых данных в postgresql

In [80]:
import pandas as pd
from sqlalchemy import create_engine, text, exc
import logging
from datetime import datetime

def add_only_new_data(df, table_name='movies', unique_columns=['title']):

    DB_CONFIG = {
        'dbname': 'admin',
        'user': 'admin',
        'password': None,
        'host': 'localhost',
        'port': '5432'
    }
    
    logger = logging.getLogger(__name__)
    
    try:
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
        engine = create_engine(connection_string)

        existing_query = f"""
            SELECT DISTINCT {', '.join(unique_columns)} 
            FROM {table_name}
        """
        
        try:
            existing_df = pd.read_sql(existing_query, engine)
        except exc.ProgrammingError:
            logger.info(f"Таблица {table_name} не существует. Создаем новую.")
            df.to_sql(table_name, engine, if_exists='replace', index=False)
            logger.info(f"Добавлено {len(df)} новых записей")
            engine.dispose()
            return len(df)

        if not existing_df.empty:
            existing_df['key'] = existing_df[unique_columns].astype(str).agg('|'.join, axis=1)
            df['key'] = df[unique_columns].astype(str).agg('|'.join, axis=1)

            new_records = df[~df['key'].isin(existing_df['key'])]
            new_records = new_records.drop('key', axis=1)
        else:
            new_records = df

        if not new_records.empty:
            new_records.to_sql(table_name, engine, if_exists='append', index=False)
            logger.info(f"Добавлено {len(new_records)} новых записей, {len(df)} всего")

            return {
                'total_processed': len(df),
                'new_added': len(new_records),
                'existing_skipped': len(df) - len(new_records),
                'new_records': new_records
            }
        else:
            logger.info("Все записи уже существуют в базе данных")
            return {
                'total_processed': len(df),
                'new_added': 0,
                'existing_skipped': len(df),
                'new_records': pd.DataFrame()
            }
        
    except Exception as e:
        logger.error(f"Ошибка при добавлении новых данных: {e}")
        raise
    finally:
        if 'engine' in locals():
            engine.dispose()

для проверки добавила фильм

In [81]:
new_film = {'title': 'test film', 'title_secondary': 'test film', 'description': ' ', 'rating': '7.4', 'age': '16+', 'genre': ' ', 'producer': ' ', 'actors': '', 'info_films': 'Продолжительность, 1 час 52 минут.'}
films_xpath_df = pd.concat([films_xpath_df, pd.DataFrame([new_film])], ignore_index=True)

In [82]:
result = add_only_new_data(
    films_xpath_df, 
    table_name='movies', 
    unique_columns=['title', 'title_secondary']
)

INFO:__main__:Добавлено 1 новых записей, 151 всего


вывод фильмов с рейтингом больше 8.0

<img src="/Users/admin/Desktop/WebScraping/Снимок экрана 2025-12-10 в 15.18.38.png">

### MangoDB

In [9]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [87]:
with open('uri.txt','r') as fil:
    uri = fil.readline()

In [90]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [91]:
client.list_database_names()

['sample_mflix', 'admin', 'local']

In [92]:
db = client['sample_mflix']
collection = db["movies_df"]

In [93]:
collection.delete_many({})
collection.insert_many(films_xpath_df.to_dict("records"))

InsertManyResult([ObjectId('693ac9552bec0e27677164a5'), ObjectId('693ac9552bec0e27677164a6'), ObjectId('693ac9552bec0e27677164a7'), ObjectId('693ac9552bec0e27677164a8'), ObjectId('693ac9552bec0e27677164a9'), ObjectId('693ac9552bec0e27677164aa'), ObjectId('693ac9552bec0e27677164ab'), ObjectId('693ac9552bec0e27677164ac'), ObjectId('693ac9552bec0e27677164ad'), ObjectId('693ac9552bec0e27677164ae'), ObjectId('693ac9552bec0e27677164af'), ObjectId('693ac9552bec0e27677164b0'), ObjectId('693ac9552bec0e27677164b1'), ObjectId('693ac9552bec0e27677164b2'), ObjectId('693ac9552bec0e27677164b3'), ObjectId('693ac9552bec0e27677164b4'), ObjectId('693ac9552bec0e27677164b5'), ObjectId('693ac9552bec0e27677164b6'), ObjectId('693ac9552bec0e27677164b7'), ObjectId('693ac9552bec0e27677164b8'), ObjectId('693ac9552bec0e27677164b9'), ObjectId('693ac9552bec0e27677164ba'), ObjectId('693ac9552bec0e27677164bb'), ObjectId('693ac9552bec0e27677164bc'), ObjectId('693ac9552bec0e27677164bd'), ObjectId('693ac9552bec0e27677164

функция для добавления новых данных в MangoDB

In [94]:
from pymongo import MongoClient, UpdateOne
from typing import List, Dict, Union
import pandas as pd

def insert_only_new_data(
    collection,
    data: Union[pd.DataFrame, List[Dict]],
    unique_keys: List[str],
    update_existing: bool = False
) -> Dict:

    if isinstance(data, pd.DataFrame):
        records = data.replace({pd.NA: None}).to_dict('records')
    else:
        records = data
    
    if not records:
        return {"inserted": 0, "updated": 0, "skipped": 0}

    try:
        collection.create_index(
            [(key, 1) for key in unique_keys],
            unique=True,
            background=True
        )
    except:
        pass 
    
    inserted_count = 0
    updated_count = 0
    skipped_count = 0
    
    for record in records:
        query = {key: record[key] for key in unique_keys if key in record}
        
        if not query: 
            skipped_count += 1
            continue
            
        try:
            if update_existing:
                result = collection.update_one(
                    query,
                    {"$set": record},
                    upsert=True
                )
                if result.upserted_id:
                    inserted_count += 1
                elif result.modified_count:
                    updated_count += 1
            else:
                result = collection.insert_one(record)
                if result.inserted_id:
                    inserted_count += 1
        except Exception as e:
            if "duplicate key" in str(e).lower():
                if not update_existing:
                    skipped_count += 1
            else:
                print(f"Ошибка при обработке записи: {e}")
                skipped_count += 1
    
    return {
        "inserted": inserted_count,
        "updated": updated_count,
        "skipped": skipped_count,
        "total_processed": len(records)
    }

для проверки добавила фильм

In [95]:
new_film = {'title': 'test film 2', 'title_secondary': 'test film 2', 'description': ' ', 'rating': '7.4', 'age': '16+', 'genre': ' ', 'producer': ' ', 'actors': '', 'info_films': 'Продолжительность, 1 час 52 минут.'}
films_xpath_df = pd.concat([films_xpath_df, pd.DataFrame([new_film])], ignore_index=True)

In [96]:
result = insert_only_new_data(
    collection=collection,
    data=films_xpath_df,
    unique_keys=["title"]
)

print(f"""
Результат:
• Добавлено новых: {result['inserted']}
• Пропущено (дубликаты): {result['skipped']}
• Всего обработано: {result['total_processed']}
""")


Результат:
• Добавлено новых: 1
• Пропущено (дубликаты): 151
• Всего обработано: 152



вывод фильмов с рейтингом больше 8.0

In [97]:
import pandas as pd
from pymongo import MongoClient
from typing import Optional, List, Dict, Any

def find_high_ratings(
    collection,
    rating_field: str = "rating",
    min_rating: float = 8.0,
    projection: Optional[Dict] = None,
    sort_desc: bool = True
) -> pd.DataFrame:

        query = {
            rating_field: {"$gt": min_rating, "$exists": True}
        }

        sort_order = [(rating_field, -1 if sort_desc else 1)]

        cursor = collection.find(
            filter=query,
            projection=projection
        ).sort(sort_order)

        results = list(cursor)
        
        if not results:
            print(f"Документов с {rating_field} > {min_rating} не найдено")
            return pd.DataFrame()

        df = pd.DataFrame(results)
        
        if '_id' in df.columns:
            df['_id'] = df['_id'].astype(str)

        if rating_field in df.columns:
            df[f'{rating_field}_formatted'] = df[rating_field].apply(
                lambda x: f"{x:.1f}" if isinstance(x, (int, float)) else str(x)
            )

        print(f"Найдено {len(df)} документов с {rating_field} > {min_rating}")
       
        return df

In [98]:
high_rated_movies = find_high_ratings(
    collection=collection,
    rating_field="rating",
    min_rating=8.0,
)
high_rated_movies

Найдено 9 документов с rating > 8.0


,_id,title,title_secondary,description,rating,age,genre,producer,actors,info_films,key,rating_formatted
0,693ac9552bec0e2767716506,Большая земля,,,9.3,,драма,Юлия Трофимова,"Анастасия Куимова, Артем Быстров, Евгений Хари...","Страна, Россия, Год выпуска, 2025, Премьера в ...",Большая земля|,9.3
1,693ac9552bec0e27677164d1,Синистер. Первое проклятие,The Confession,,9.1,18+,ужасы,Уилл Кэнон,"Скотт Мехловиц, Vince Eisenson, Ron Fallica, Z...","Страна, США, Продолжительность, 1 час 27 минут...",Синистер. Первое проклятие|The Confession,9.1
2,693ac9552bec0e27677164ed,Здесь был Юра,,,9.1,,драма,Сергей Малкин,"Константин Хабенский, Кузьма Котрелев, Даша Ко...","Страна, Россия, Продолжительность, 1 час 36 ми...",Здесь был Юра|,9.1
3,693ac9552bec0e27677164a6,Зверополис 2,Zootopia 2,"Джуди и Ник вновь объединяются, чтобы раскрыть...",8.8,6+,"приключения, анимация, комедия",Джаред Буш,"Джейсон Бэйтман, Кинта Брансон, Фортун Феймсте...","Страна, США, Продолжительность, 1 час 47 минут...",Зверополис 2|Zootopia 2,8.8
4,693ac9552bec0e27677164c7,Оно. Первое пришествие,Diabolic,,8.6,18+,ужасы,Дэниэл Дж. Филлипс,"Джон Харлан Ким, Элизабет Каллен, Luca Asta Sa...","Страна, Австралия, Продолжительность, 1 час 35...",Оно. Первое пришествие|Diabolic,8.6
5,693ac9552bec0e27677164c0,Аватар: Пламя и пепел,Avatar: Fire and Ash,Третья часть мегафраншизы Джеймса Кэмерона о р...,8.5,18+,"фантастика, фэнтези, боевик, приключения",Джеймс Кэмерон,"Зои Салдана, Сигурни Уивер, Джемейн Клемент, М...","Страна, США, Продолжительность, 3 часа 15 мину...",Аватар: Пламя и пепел|Avatar: Fire and Ash,8.5
6,693ac9552bec0e27677164a8,Агент времени: Глава Инду. Фильм,Link Click: Bridon Arc - The Movie,Полнометражное продолжение популярного китайск...,8.4,18+,"приключения, анимация, криминал",Haolin Li,,"Страна, Китай, Продолжительность, 1 час 41 мин...",Агент времени: Глава Инду. Фильм|Link Click: B...,8.4
7,693ac9552bec0e27677164b9,Волчьи дети Амэ и Юки,Ookami Kodomo no Ame to Yuki,Признанное фэнтезийное аниме Мамору Хосоды о в...,8.2,12+,"фэнтези, анимация, аниме",Мамору Хосода,"Кумико Асо, Джош Грелл, Мэгуми Хаясибара, Таку...","Страна, Япония, Продолжительность, 1 час 57 ми...",Волчьи дети Амэ и Юки|Ookami Kodomo no Ame to ...,8.2
8,693ac9552bec0e27677164ba,Пятый элемент,The Fifth Element,Бывший секретный агент сталкивается с иноплане...,8.1,18+,"приключения, триллер, боевик, фантастика, мело...",Люк Бессон,"Брюс Уиллис, Милла Йовович, Гэри Олдман, Брайо...","Страна, Франция, Продолжительность, 2 часа 6 м...",Пятый элемент|The Fifth Element,8.1
